In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score
import os
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap


/var/folders/zg/ds4nwmvx3kq28hn4r4zwbbw80000gp/T/ipykernel_80383/3827359723.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ds_version = "lndw"
metadata_file = "metadata.csv"
combined_file = "prediction_lndw.csv"
pred_file = "effnet_surface_quality_prediction-lndw-20240621_161858.csv"
data_path = os.path.join("..", "data", ds_version)

In [14]:
pred = pd.DataFrame()

print(f"reading {pred_file}")
pred = pd.read_csv(os.path.join(data_path, pred_file))
pred.rename(columns={"Image": "image_id",  "Level_0": "type_pred", "Level_1": "quality_label_pred"}, inplace=True)

quality_pred = pred[pred.Level == "quality"][["image_id", "Prediction", "quality_label_pred", "type_pred"]]
quality_pred.rename(columns={"Prediction": "quality_float_pred"}, inplace=True)

asphalt_pred = pred[(pred.Level == "type") & (pred.type_pred == "asphalt")][["image_id", "Prediction"]]
asphalt_pred.rename(columns={"Prediction": "asphalt_pred"}, inplace=True)
concrete_pred = pred[(pred.Level == "type") & (pred.type_pred == "concrete")][["image_id", "Prediction"]]
concrete_pred.rename(columns={"Prediction": "concrete_pred"}, inplace=True)
paving_stones_pred = pred[(pred.Level == "type") & (pred.type_pred == "paving_stones")][["image_id", "Prediction"]]
paving_stones_pred.rename(columns={"Prediction": "paving_stones_pred"}, inplace=True)
sett_pred = pred[(pred.Level == "type") & (pred.type_pred == "sett")][["image_id", "Prediction"]]
sett_pred.rename(columns={"Prediction": "sett_pred"}, inplace=True)
unpaved_pred = pred[(pred.Level == "type") & (pred.type_pred == "unpaved")][["image_id", "Prediction"]]
unpaved_pred.rename(columns={"Prediction": "unpaved_pred"}, inplace=True)

# type_pred = pred[pred.Level == "type"][["image_id", "Prediction", "type_pred", "is_in_validation"]]
# type_pred.rename(columns={"Prediction": "class_prob"}, inplace=True)
# idx = type_pred.groupby("image_id")["class_prob"].idxmax()
# type_pred = type_pred.loc[idx]
# pred = quality_pred.set_index("image_id").join(type_pred.set_index("image_id"), lsuffix="_quality", rsuffix="_type")

reading effnet_surface_quality_prediction-lndw-20240621_161858.csv


In [15]:
asphalt_pred

,image_id,asphalt_pred
0,1001419624308587,0.999409
5,1001712924422413,0.000172
10,1001853784055707,0.999911
15,1001855950352623,0.000008
20,1002859683992939,0.997848
...,...,...
20305,993987041344973,0.001576
20310,994495108188796,0.758562
20315,994580034492940,0.000035
20320,994744601331046,0.002794


In [16]:
df = quality_pred.set_index("image_id").join(
    asphalt_pred.set_index("image_id"), how="left"
    ).join(
    concrete_pred.set_index("image_id"), how="left"
    ).join(
    paving_stones_pred.set_index("image_id"), how="left"
    ).join(
    sett_pred.set_index("image_id"), how="left"
    ).join(
    unpaved_pred.set_index("image_id"), how="left"
    )

In [18]:
print(df)

                  quality_float_pred quality_label_pred type_pred  \
image_id                                                            
1001419624308587            2.113812               good   asphalt   
1001853784055707            0.742298          excellent   asphalt   
1002859683992939            1.742429               good   asphalt   
1007098503152985            0.862858          excellent   asphalt   
1007523906448355            1.583960               good   asphalt   
...                              ...                ...       ...   
969102130527611             2.748043       intermediate   unpaved   
972064743529126             3.100275       intermediate   unpaved   
991410295002276             3.475808       intermediate   unpaved   
993107898353925             3.849031                bad   unpaved   
994744601331046             3.770034                bad   unpaved   

                  asphalt_pred  concrete_pred  paving_stones_pred  \
image_id                         

In [19]:
metadata = pd.read_csv(os.path.join(data_path, metadata_file))
metadata.rename(columns={"mapillary_image_id": "image_id", "surface_type": "type_true", "surface_quality": "quality_label_true"}, inplace=True)
# annot["quality_float_true"] = annot["quality_label_true"].map({"excellent": 1, "good": 2, "intermediate": 3, "bad": 4, "very_bad": 5})

In [25]:
df = df.join(
    metadata.set_index("image_id"), how="left")


ValueError: columns overlap but no suffix specified: Index(['user_id', 'user_name', 'captured_at', 'longitude', 'latitude',
       'type_true', 'quality_label_true'],
      dtype='object')

In [24]:
df.to_csv(os.path.join(data_path, combined_file), index=True)

In [22]:
df

,quality_float_pred,quality_label_pred,type_pred,asphalt_pred,concrete_pred,paving_stones_pred,sett_pred,unpaved_pred,user_id,user_name,captured_at,longitude,latitude,type_true,quality_label_true
image_id,,,,,,,,,,,,,,,
1001419624308587,2.113812,good,asphalt,0.999409,0.000047,1.925988e-05,7.009975e-05,4.539925e-04,100232345559807,carlheinz,1698249177500,13.457938,52.512914,asphalt,good
1001853784055707,0.742298,excellent,asphalt,0.999911,0.000060,5.032652e-06,2.106234e-05,2.814698e-06,102627028648372,kartonage,1652268883000,13.389880,52.546260,asphalt,excellent
1002859683992939,1.742429,good,asphalt,0.997848,0.002150,1.011511e-06,2.040124e-07,1.614338e-08,107787624790453,aa_blaablaa,1645696377683,13.350672,52.486629,asphalt,good
1007098503152985,0.862858,excellent,asphalt,0.999935,0.000052,5.182736e-06,7.341004e-06,5.135955e-07,102627028648372,kartonage,1552202319000,13.513892,52.463599,NaN,NaN
1007523906448355,1.583960,good,asphalt,0.996514,0.003470,1.410846e-06,1.500023e-05,1.720438e-07,110282201203825,eserte,1605955691712,13.515211,52.462220,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969102130527611,2.748043,intermediate,unpaved,0.451487,0.023161,8.249066e-05,4.709266e-05,5.252219e-01,106378651601030,karlich,1586270175274,13.543020,52.466452,NaN,NaN
972064743529126,3.100275,intermediate,unpaved,0.058543,0.000534,9.915962e-02,2.195735e-02,8.198055e-01,110282201203825,eserte,1591023388514,13.533134,52.468861,NaN,NaN
991410295002276,3.475808,intermediate,unpaved,0.001879,0.000770,9.838669e-05,3.550718e-05,9.972167e-01,110282201203825,eserte,1591023202506,13.546937,52.468763,NaN,NaN


In [30]:
df.groupby(["type_true", "quality_label_true"]).size()

type_true      quality_label_true
asphalt        bad                    238
               excellent              791
               good                  1407
               intermediate           768
concrete       bad                     58
               excellent              310
               good                   340
               intermediate           248
paving_stones  bad                     70
               excellent              360
               good                   989
               intermediate           505
sett           bad                    526
               good                   128
               intermediate           677
unpaved        bad                    339
               intermediate           305
               very_bad               287
dtype: int64